# SciPy Sparse Matrices

sparse matrix construction, sparse solvers, graph operations, and ranking/similarity use cases.

In [1]:
import numpy as np
from scipy import sparse
from scipy.sparse import linalg as spla
from scipy.sparse import csgraph

np.set_printoptions(precision=4, suppress=True)

## 1. Building Sparse Matrices
Sparse formats store only nonzero entries and significantly reduce memory in high-dimensional problems.

In [2]:
rng = np.random.default_rng(4)

num_docs = 500
num_terms = 6000
nnz = 18000
rows = rng.integers(0, num_docs, size=nnz)
cols = rng.integers(0, num_terms, size=nnz)
vals = rng.integers(1, 5, size=nnz)
term_doc = sparse.coo_matrix((vals, (rows, cols)), shape=(num_docs, num_terms)).tocsr()

print("Example 1: Term-document matrix")
print(f"Shape: {term_doc.shape}")
print(f"Stored values: {term_doc.nnz}")
print(f"Density: {term_doc.nnz / (term_doc.shape[0] * term_doc.shape[1]):.6f}")

dense_bytes = np.zeros(term_doc.shape, dtype=np.float64).nbytes
sparse_bytes = term_doc.data.nbytes + term_doc.indices.nbytes + term_doc.indptr.nbytes
print(f"Dense memory estimate: {dense_bytes / 1024 ** 2:.2f} MB")
print(f"Sparse memory estimate: {sparse_bytes / 1024 ** 2:.2f} MB")

users = 1000
items = 800
ratings_nnz = 12000
u = rng.integers(0, users, size=ratings_nnz)
i = rng.integers(0, items, size=ratings_nnz)
r = rng.uniform(1, 5, size=ratings_nnz)
ratings = sparse.csr_matrix((r, (u, i)), shape=(users, items))

print()
print("Example 2: User-item interaction matrix")
print(f"Shape: {ratings.shape}")
print(f"Stored ratings: {ratings.nnz}")

Example 1: Term-document matrix
Shape: (500, 6000)
Stored values: 17936
Density: 0.005979
Dense memory estimate: 22.89 MB
Sparse memory estimate: 0.21 MB

Example 2: User-item interaction matrix
Shape: (1000, 800)
Stored ratings: 11915


## 2. Sparse Linear Algebra
Sparse solvers efficiently handle large linear systems common in engineering and ML workflows.

In [3]:
n = 900
main = 4 * np.ones(n)
off = -1 * np.ones(n - 1)
A = sparse.diags([off, main, off], offsets=[-1, 0, 1], format='csr')
b = np.ones(n)

x_direct = spla.spsolve(A, b)
res_direct = np.linalg.norm(A @ x_direct - b)

print("Example 1: Direct sparse solve")
print(f"Residual norm: {res_direct:.3e}")

x_cg, info = spla.cg(A, b, rtol=1e-8, maxiter=3000)
res_cg = np.linalg.norm(A @ x_cg - b)

print()
print("Example 2: Conjugate gradient solve")
print(f"CG info flag: {info}")
print(f"Residual norm: {res_cg:.3e}")

Example 1: Direct sparse solve
Residual norm: 9.615e-16

Example 2: Conjugate gradient solve
CG info flag: 0
Residual norm: 8.972e-08


## 3. Sparse Graph Algorithms
Sparse graph methods support route analysis and community structure discovery.

In [4]:
road = sparse.csr_matrix([
    [0, 7, 0, 0, 0, 14],
    [7, 0, 10, 15, 0, 0],
    [0, 10, 0, 11, 0, 2],
    [0, 15, 11, 0, 6, 0],
    [0, 0, 0, 6, 0, 9],
    [14, 0, 2, 0, 9, 0]
], dtype=float)

dist, _ = csgraph.shortest_path(road, directed=False, return_predecessors=True)

print("Example 1: Road shortest-path distance")
print(f"Distance from node 0 to node 4: {dist[0, 4]:.2f}")

social = sparse.csr_matrix([
    [0, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]
], dtype=int)

n_comp, labels = csgraph.connected_components(social, directed=False)

print()
print("Example 2: Social graph components")
print(f"Connected components: {n_comp}")
print(f"Labels: {labels.tolist()}")

Example 1: Road shortest-path distance
Distance from node 0 to node 4: 23.00

Example 2: Social graph components
Connected components: 3
Labels: [0, 0, 0, 1, 1, 1, 2]


## 4. Ranking and Similarity with Sparse Operations
Sparse operations can compute network ranking and item similarity for recommendation systems.

In [5]:
link = sparse.csr_matrix([
    [0, 1, 1, 0],
    [0, 0, 1, 1],
    [1, 0, 0, 1],
    [1, 0, 0, 0]
], dtype=float)

out_degree = np.asarray(link.sum(axis=1)).ravel()
transition = sparse.diags(1 / np.maximum(out_degree, 1)) @ link
rank = np.full(4, 0.25)
for _ in range(60):
    rank = 0.15 / 4 + 0.85 * (transition.T @ rank)

print("Example 1: PageRank-style scoring")
print("Scores:", np.round(rank, 4))

rng = np.random.default_rng(18)
mat = sparse.random(120, 90, density=0.12, random_state=18, data_rvs=lambda k: rng.uniform(1, 5, size=k)).tocsr()
sim = (mat.T @ mat).tocsr()
sim.setdiag(0)
row10 = sim.getrow(10).toarray().ravel()
nearest = np.argsort(row10)[-5:][::-1]

print()
print("Example 2: Item similarity lookup")
print(f"Top related items for item 10: {nearest.tolist()}")

Example 1: PageRank-style scoring
Scores: [0.3374 0.1809 0.2578 0.2239]

Example 2: Item similarity lookup
Top related items for item 10: [26, 2, 56, 84, 11]
